<img src='https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQ-VfNtOyJbsaxu43Kztf_cv1mgBG6ZIQZEVw&usqp=CAU'>

# Procesamiento de Lenguaje Natural

## Taller #9: Modelado de temas
`Fecha de entrega: Mayo 6, 2021. (Antes del inicio de la próxima clase).`

`Modo de entrega: Subir link de GitHub al aula virtual.`

In [9]:
import re
import pandas as pd 
from pprint import pprint

from nltk.corpus import stopwords
stopwords = stopwords.words('spanish')

!pip install -U pandas-profiling
!pip install pyLDAvis
import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Requirement already up-to-date: pandas-profiling in c:\users\andre\anaconda3\lib\site-packages (2.12.0)


In [10]:
# Cargar datos
path = 'reviews_vidjew_es (1).csv'
data = pd.read_csv(path)
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games


###  `[12 pts]` Punto 1: Hacer pre-procesamiento del texto

In [11]:
def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_]+", " ", texto)
    texto = [palabra for palabra in texto.split() if palabra not in stopwords]
    return texto

In [12]:
data['pre-procesado'] = data['review_body'].apply(lambda texto: pre_procesado(texto))
data.head()

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category,pre-procesado
0,es_0825565,product_es_0370490,reviewer_es_0174781,3,"Buen. Buena calidad, y buena presentación.",Contenta,es,jewelry,"[buen, buena, calidad, buena, presentación]"
1,es_0227934,product_es_0354224,reviewer_es_0411613,3,"Un producto a perfecto, para salir de casa con...",Versatilidad,es,video_games,"[producto, perfecto, salir, casa, nintendo, sw..."
2,es_0468601,product_es_0665460,reviewer_es_0348315,1,No funciona con Nintendo Switch. No hay forma ...,Decepción absoluta,es,video_games,"[funciona, nintendo, switch, forma, emparejarl..."
3,es_0814494,product_es_0692692,reviewer_es_0951508,5,"Recomendado, los utilizo para pc y no me dan n...",Auriculares Pecham ps4,es,video_games,"[recomendado, utilizo, pc, dan, ningún, proble..."
4,es_0206329,product_es_0728826,reviewer_es_0493255,4,El cable funciona bien podria ser un poco mas ...,Perfecto,es,video_games,"[cable, funciona, bien, podria, ser, mas, larg..."


###  `[10 pts]` Punto 2: Modelo de LDA

In [22]:
dictionary = Dictionary(data['pre-procesado'].values)
dictionary

#Filtrar palabras
dictionary.filter_extremes(no_below=5, no_above=0.5)

#Corpus

corpus = [dictionary.doc2bow(text) for text in data['pre-procesado'].values] 
#Entrenar modelo

model = LdaModel(corpus = corpus, id2word = dictionary, num_topics = 15, passes=500)

#imprimir modelo

model.print_topics(num_words=10)



[(0,
  '0.047*"si" + 0.033*"esperaba" + 0.028*"solo" + 0.023*"versión" + 0.020*"caro" + 0.020*"español" + 0.020*"mando" + 0.019*"así" + 0.018*"xbox" + 0.016*"play"'),
 (1,
  '0.177*"juego" + 0.026*"entrega" + 0.026*"perfectamente" + 0.025*"gran" + 0.023*"funda" + 0.022*"ser" + 0.020*"viene" + 0.019*"pena" + 0.018*"fácil" + 0.018*"servicio"'),
 (2,
  '0.084*"llegado" + 0.048*"día" + 0.035*"recibido" + 0.034*"pedido" + 0.032*"producto" + 0.027*"entregado" + 0.027*"tiempo" + 0.026*"llegar" + 0.026*"todavía" + 0.025*"sido"'),
 (3,
  '0.055*"si" + 0.035*"bonita" + 0.032*"mas" + 0.030*"bien" + 0.028*"juegos" + 0.025*"uso" + 0.024*"ahora" + 0.024*"caja" + 0.023*"queda" + 0.017*"puede"'),
 (4,
  '0.056*"llego" + 0.050*"mas" + 0.032*"juego" + 0.032*"hijo" + 0.029*"puedo" + 0.025*"q" + 0.021*"nunca" + 0.019*"producto" + 0.018*"tiempo" + 0.017*"decir"'),
 (5,
  '0.072*"bonito" + 0.048*"foto" + 0.035*"queda" + 0.033*"tal" + 0.028*"grande" + 0.027*"pequeño" + 0.026*"parece" + 0.023*"ve" + 0.020*"au

###  `[25 pts]` Punto 3: Visualización de LDA

In [23]:
lda_display = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
pyLDAvis.display(lda_display)

###  `[3 pts]` Preguntas

- ¿Qué conclusiones puede sacar de los resultados de los últimos tres talleres, usando diferentes modelos pero los mismos datos?

Lo que identifico es que de acuerdo a la necesidad se puede ir ajustando el modelo, sin embargo cuando hacemos LDA depende mucho de la observaciín subjetiva del analista de datos es decir de los paraemtros que le lo cual puede sesgar un poco el modelo.